In [2]:
import pandas as pd
import os

from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline
from pyspark.sql import functions as fn, Row
from pyspark import sql

import matplotlib.pyplot as plt
import pandas as pd

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

from datetime import datetime
from pyspark.sql.functions import col,udf
from pyspark.sql.types import DateType
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [4]:
import pandas as pd
fulldata=pd.read_csv("C:/Users/SHIVANSHI/IST718 Dropbox/IST 718 Project/Taxi Data/2015-01_100k.csv")
weatherdata = pd.read_excel("C:/Users/SHIVANSHI/IST718 Dropbox/IST 718 Project/Weather Data/2015_weather.xlsx")
holidaysdata = pd.read_excel("C:/Users/SHIVANSHI/IST718 Dropbox/IST 718 Project/Holiday Data/holidays.xlsx")


In [20]:

datefrompickup=pd.to_datetime(fulldata['pickup_datetime']).apply(lambda x: x.date())
datefrompickup.head()
fulldata['pickup_date']=datefrompickup
fulldata.head()


,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,tolls_amount,improvement_surcharge,total_amount,pickup_zip,pickup_borough,pickup_neighborhood,dropoff_zip,dropoff_borough,dropoff_neighborhood,pickup_date
0,2,2015-01-06 11:39:29,2015-01-06 11:49:15,1,1.78,-73.999619,40.743599,1,N,-73.992203,...,0.0,0.3,9.80,10011,Manhattan,Chelsea and Clinton,10036,Manhattan,Chelsea and Clinton,2015-01-06
1,1,2015-01-13 09:18:29,2015-01-13 09:23:40,1,2.10,-73.981956,40.778290,1,N,-73.962173,...,0.0,0.3,10.75,10023,Manhattan,Upper West Side,10024,Manhattan,Upper West Side,2015-01-13
2,2,2015-01-16 07:15:44,2015-01-16 07:26:42,1,2.33,-73.991188,40.742226,1,N,-73.981613,...,0.0,0.3,11.80,10010,Manhattan,Gramercy Park and Murray Hill,10019,Manhattan,Chelsea and Clinton,2015-01-16
3,1,2015-01-23 11:56:05,2015-01-23 12:13:20,1,2.10,-73.959297,40.763336,1,N,-73.979996,...,0.0,0.3,13.30,10065,Manhattan,Upper East Side,10023,Manhattan,Upper West Side,2015-01-23
4,1,2015-01-24 10:11:48,2015-01-24 10:18:32,2,0.90,-73.971832,40.764751,1,N,-73.984047,...,0.0,0.3,7.30,10065,Manhattan,Upper East Side,10019,Manhattan,Chelsea and Clinton,2015-01-24


In [25]:
mergeddata=fulldata.join(weatherdata,lsuffix='_fulldata', rsuffix='_weatherdata')
mergeddata.head()


,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,pickup_neighborhood,dropoff_zip,dropoff_borough,dropoff_neighborhood,pickup_date_fulldata,pickup_date_weatherdata,avg_temp_C,Rain,Fog,Snow
0,2,2015-01-06 11:39:29,2015-01-06 11:49:15,1,1.78,-73.999619,40.743599,1,N,-73.992203,...,Chelsea and Clinton,10036,Manhattan,Chelsea and Clinton,2015-01-06,2015-01-01,1.0,NaN,NaN,NaN
1,1,2015-01-13 09:18:29,2015-01-13 09:23:40,1,2.10,-73.981956,40.778290,1,N,-73.962173,...,Upper West Side,10024,Manhattan,Upper West Side,2015-01-13,2015-01-02,4.0,NaN,NaN,NaN
2,2,2015-01-16 07:15:44,2015-01-16 07:26:42,1,2.33,-73.991188,40.742226,1,N,-73.981613,...,Gramercy Park and Murray Hill,10019,Manhattan,Chelsea and Clinton,2015-01-16,2015-01-03,3.0,1.0,NaN,1.0
3,1,2015-01-23 11:56:05,2015-01-23 12:13:20,1,2.10,-73.959297,40.763336,1,N,-73.979996,...,Upper East Side,10023,Manhattan,Upper West Side,2015-01-23,2015-01-04,9.0,1.0,NaN,NaN
4,1,2015-01-24 10:11:48,2015-01-24 10:18:32,2,0.90,-73.971832,40.764751,1,N,-73.984047,...,Upper East Side,10019,Manhattan,Chelsea and Clinton,2015-01-24,2015-01-05,2.0,NaN,NaN,NaN


In [26]:
fulldata.merge(weatherdata, left_on='pickup_date',right_on='pickup_date',how='inner')

,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,pickup_borough,pickup_neighborhood,dropoff_zip,dropoff_borough,dropoff_neighborhood,pickup_date,avg_temp_C,Rain,Fog,Snow


In [11]:
weatherdata.head()

,pickup_date,avg_temp_C,Rain,Fog,Snow
0,01.01.15,1,NaN,NaN,NaN
1,02.01.15,4,NaN,NaN,NaN
2,03.01.15,3,1.0,NaN,1.0
3,04.01.15,9,1.0,NaN,NaN
4,05.01.15,2,NaN,NaN,NaN


In [23]:

weatherdata['pickup_date'] = pd.to_datetime(weatherdata['pickup_date'], format="%d.%m.%y", errors='coerce')
weatherdata.head()

,pickup_date,avg_temp_C,Rain,Fog,Snow
0,2015-01-01,1,NaN,NaN,NaN
1,2015-01-02,4,NaN,NaN,NaN
2,2015-01-03,3,1.0,NaN,1.0
3,2015-01-04,9,1.0,NaN,NaN
4,2015-01-05,2,NaN,NaN,NaN


In [ ]:
#---------------------------Merged Data Operations---------------------------------------#

In [ ]:
final_data = spark.createDataFrame(pd.read_csv('C:/Users/SHIVANSHI/IST718 Dropbox/IST 718 Project/MERGED DATA/Read_Merged_Data1.csv'))

In [3]:
sample_data = spark.createDataFrame(pd.read_csv('C:/Users/SHIVANSHI/IST718 Dropbox/IST 718 Project/MERGED DATA/Pandas_Sample.csv'))

In [4]:
sample_data.show(5)

+----------+----------+--------------------+----------+----+---+----+-------+--------+-------------------+-------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+----------+--------------+--------------------+-----------+---------------+--------------------+----------------+-------------------+-----------+------------+
|Unnamed: 0| date_only|         pickup_date|avg_temp_C|Rain|Fog|Snow|Holiday|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RateCodeID|store_and_fwd_flag| dropoff_longitude|  dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|pickup_zip|pickup_borough| pickup_neighborhood|dropoff_zip|dropoff_borough|dropoff_neighborhood|pickup_dayofweek|pickup_w

In [5]:
sample_data = sample_data.drop('Unnamed: 0','pickup_date','date_only')

In [6]:
sample_data.printSchema()

root
 |-- avg_temp_C: long (nullable = true)
 |-- Rain: long (nullable = true)
 |-- Fog: long (nullable = true)
 |-- Snow: long (nullable = true)
 |-- Holiday: long (nullable = true)
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- RateCodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_a

In [8]:
sample_data.printSchema()

root
 |-- avg_temp_C: long (nullable = true)
 |-- Rain: long (nullable = true)
 |-- Fog: long (nullable = true)
 |-- Snow: long (nullable = true)
 |-- Holiday: long (nullable = true)
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- RateCodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_a

array([ 0.02460368,  0.00459694, -0.00242413, -0.02030871, -0.01128363,
        0.00815524,  0.01416242,  0.89698299,  0.29461282, -0.02203972,
        0.04024261,  0.31885632, -0.09000388, -0.18518544,  0.03929161,
       -0.06652559,  0.64608975,  0.58869309,  0.01163316,  1.        ,
        0.36270077,  0.41295884, -0.01658767,  0.00465715,  0.02822905])

In [58]:
dummy_df = sample_data.select('*',fn.when(fn.col('pickup_borough') == 'Queens',1).otherwise(0).alias('pickup_Queens'),
                           fn.when(fn.col('pickup_borough') == 'Brooklyn',1).otherwise(0).alias('pickup_Brooklyn'),
                           fn.when(fn.col('pickup_borough') == 'Manhattan',1).otherwise(0).alias('pickup_Manhattan'),
                           fn.when(fn.col('pickup_borough') == 'Bronx',1).otherwise(0).alias('pickup_Bronx'),
                           fn.when(fn.col('dropoff_borough') == 'Queens',1).otherwise(0).alias('dropoff_Queens'),
                           fn.when(fn.col('dropoff_borough') == 'Brooklyn',1).otherwise(0).alias('dropoff_Brooklyn'),
                           fn.when(fn.col('dropoff_borough') == 'Manhattan',1).otherwise(0).alias('dropoff_Manhattan'),
                           fn.when(fn.col('dropoff_borough') == 'Bronx',1).otherwise(0).alias('dropoff_Bronx'))
dummy_df = dummy_df.drop('pickup_borough','dropoff_borough')

In [59]:
dummy_df.show(5)

+----------+----+---+----+-------+--------+-------------------+-------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+----------+--------------------+-----------+--------------------+----------------+-------------------+-----------+------------+-------------+---------------+----------------+------------+--------------+----------------+-----------------+-------------+
|avg_temp_C|Rain|Fog|Snow|Holiday|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RateCodeID|store_and_fwd_flag| dropoff_longitude|  dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|pickup_zip| pickup_neighborhood|dropoff_zip|dropoff_neighborhood|pickup_dayofweek|pickup_weekday_name|pickup_hour|

In [60]:
dummy_df = dummy_df.withColumn('pickup_datetime', fn.to_timestamp(fn.col('pickup_datetime')))
dummy_df = dummy_df.withColumn('dropoff_datetime', fn.to_timestamp(fn.col('dropoff_datetime')))

In [61]:
dummy_df.printSchema()

root
 |-- avg_temp_C: long (nullable = true)
 |-- Rain: long (nullable = true)
 |-- Fog: long (nullable = true)
 |-- Snow: long (nullable = true)
 |-- Holiday: long (nullable = true)
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- RateCodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- t

In [11]:
dummy_df.select('payment_type').distinct().show()

+------------+
|payment_type|
+------------+
|           1|
|           3|
|           2|
|           4|
+------------+



In [62]:
#base is unknown #5
dummy_df = dummy_df.select('*',fn.when(fn.col('payment_type') == '1',1).otherwise(0).alias('ptype_CCard'),
                           fn.when(fn.col('payment_type') == '2',1).otherwise(0).alias('ptype_cash'),
                           fn.when(fn.col('payment_type') == '3',1).otherwise(0).alias('ptype_nocharge'),
                           fn.when(fn.col('payment_type') == '4',1).otherwise(0).alias('ptype_dispute'),
                           fn.when(fn.col('payment_type') == '6',1).otherwise(0).alias('ptype_voidtrip'))
dummy_df = dummy_df.drop('payment_type')

In [63]:
# base is 99
dummy_df= dummy_df.select('*',fn.when(fn.col('RateCodeID')=='1',1).otherwise(0).alias('RateCode_stdrate'), fn.when(fn.col('RateCodeID')=='2',1).otherwise(0).alias('RateCode_JFK'),
                          fn.when(fn.col('RateCodeID')=='3',1).otherwise(0).alias('RateCode_Newark'), fn.when(fn.col('RateCodeID')=='4',1).otherwise(0).alias('RateCode_NW'),
                         fn.when(fn.col('RateCodeID')=='5',1).otherwise(0).alias('RateCode_Negotiated'), fn.when(fn.col('RateCodeID')=='6',1).otherwise(0).alias('RateCode_Group'))
dummy_df = dummy_df.drop('RateCodeID')

In [64]:
len(dummy_df.columns) #45

49

In [65]:
dummy_df = dummy_df.withColumn('store_and_fwd_flag',fn.when(fn.col('store_and_fwd_flag')== 'Y',1).otherwise(0))

In [66]:
#base is 2
dummy_df = dummy_df.withColumn('VendorID',fn.when(fn.col('VendorID')=='1',1).otherwise(0))

In [67]:
dummy_df1 = dummy_df
dummy_df1 = dummy_df1.sample(False, 1.0, 42)

In [68]:
dummy_df1.show(2)

+----------+----+---+----+-------+--------+-------------------+-------------------+---------------+-------------+------------------+------------------+------------------+------------------+-----------------+-----------+-----+-------+----------+------------+---------------------+------------+----------+-------------------+-----------+--------------------+----------------+-------------------+-----------+------------+-------------+---------------+----------------+------------+--------------+----------------+-----------------+-------------+-----------+----------+--------------+-------------+--------------+----------------+------------+---------------+-----------+-------------------+--------------+
|avg_temp_C|Rain|Fog|Snow|Holiday|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|store_and_fwd_flag| dropoff_longitude| dropoff_latitude|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|picku

In [70]:
#dummy_df1.select('pickup_datetime','weekday').show(2)

In [71]:
dummy_df1.show(2)

+----------+----+---+----+-------+--------+-------------------+-------------------+---------------+-------------+------------------+------------------+------------------+------------------+-----------------+-----------+-----+-------+----------+------------+---------------------+------------+----------+-------------------+-----------+--------------------+----------------+-------------------+-----------+------------+-------------+---------------+----------------+------------+--------------+----------------+-----------------+-------------+-----------+----------+--------------+-------------+--------------+----------------+------------+---------------+-----------+-------------------+--------------+
|avg_temp_C|Rain|Fog|Snow|Holiday|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|store_and_fwd_flag| dropoff_longitude| dropoff_latitude|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|picku

In [79]:
from pyspark.sql.functions import unix_timestamp, from_unixtime, date_format

df1=dummy_df1.select(unix_timestamp(dummy_df1.pickup_datetime).alias('ut'))\
  .select(from_unixtime('ut').alias('dty'))\
  .select(date_format('dty', 'M/d/yyyy').alias('Pickup_only_Date'),
          date_format('dty', 'h:m:s a').alias('Pickup_only_Time'),'*')

df1.show(2)

#df2=df1.select(unix_timestamp(df1.dropoff_datetime).alias('ut'))\
#  .select(from_unixtime('ut').alias('dty'))\
#  .select(date_format('dty', 'M/d/yyyy').alias('Dropoff_only_Date'),
#          date_format('dty', 'h:m:s a').alias('Dropoff_only_Time'),'*')
  


+----------------+----------------+-------------------+
|Pickup_only_Date|Pickup_only_Time|                dty|
+----------------+----------------+-------------------+
|      12/27/2015|     12:57:54 AM|2015-12-27 00:57:54|
|      11/12/2015|       8:27:0 AM|2015-11-12 08:27:00|
+----------------+----------------+-------------------+
only showing top 2 rows



In [82]:
from pyspark.sql import functions as F
timeFmt = "yyyy-MM-dd'T'HH:mm:ss.SSS"
timeDiff = (F.unix_timestamp(dummy_df1.dropoff_datetime, format=timeFmt)
            - F.unix_timestamp(dummy_df1.pickup_datetime, format=timeFmt))
#dummy_df1 = dummy_df1.wfrom pyspark.sql import functions as F
timeFmt = "yyyy-MM-dd'T'HH:mm:ss.SSS"
timeDiff = (F.unix_timestamp(dummy_df1.dropoff_datetime, format=timeFmt)
            - F.unix_timestamp(dummy_df1.pickup_datetime, format=timeFmt))
dummy_df1 = dummy_df1.withColumn("Duration", timeDiff)

In [83]:
dummy_df1.show(2)

+----------+----+---+----+-------+--------+-------------------+-------------------+---------------+-------------+------------------+------------------+------------------+------------------+-----------------+-----------+-----+-------+----------+------------+---------------------+------------+----------+-------------------+-----------+--------------------+----------------+-------------------+-----------+------------+-------------+---------------+----------------+------------+--------------+----------------+-----------------+-------------+-----------+----------+--------------+-------------+--------------+----------------+------------+---------------+-----------+-------------------+--------------+--------+
|avg_temp_C|Rain|Fog|Snow|Holiday|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|store_and_fwd_flag| dropoff_longitude| dropoff_latitude|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amo

In [84]:
dummy_df1.printSchema()

root
 |-- avg_temp_C: long (nullable = true)
 |-- Rain: long (nullable = true)
 |-- Fog: long (nullable = true)
 |-- Snow: long (nullable = true)
 |-- Holiday: long (nullable = true)
 |-- VendorID: integer (nullable = false)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- store_and_fwd_flag: integer (nullable = false)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_zip: long (nullable = true

In [2]:
len(dummy_df1.columns)

NameError: name 'dummy_df1' is not defined

In [89]:
va = feature.VectorAssembler(inputCols=['avg_temp_C','Rain','Fog','Snow','Holiday','VendorID',
                                        'passenger_count', 'trip_distance','store_and_fwd_flag',
                                        'pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude',
                                        'extra','mta_tax','tip_amount','tolls_amount','improvement_surcharge',
                                        'total_amount','pickup_zip','dropoff_zip',
                                        'pickup_dayofweek', 'pickup_hour', 'pickup_month','pickup_Queens','pickup_Brooklyn',
                                        'pickup_Manhattan','pickup_Bronx','dropoff_Queens','dropoff_Brooklyn',
                                        'dropoff_Manhattan','dropoff_Bronx','ptype_CCard','ptype_cash','ptype_nocharge',
                                        'ptype_dispute','ptype_voidtrip','RateCode_stdrate','RateCode_JFK',
                                        'RateCode_Newark','RateCode_NW','RateCode_Negotiated','RateCode_Group',
                                        'Duration'] , outputCol='features')

In [90]:
features_sample_df = va.transform(dummy_df1)

from pyspark.ml.stat import Correlation
r1 = Correlation.corr(features_sample_df, 'features').head()


In [92]:
#print("Pearson correlation matrix:\n" + str(r1[0]))
#''avg_temp_C','Rain','Fog','Snow','Holiday','VendorID',
#                                        'passenger_count', 'trip_distance','store_and_fwd_flag',
 #                                       'pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude',
 #                                       'extra','mta_tax','tip_amount','tolls_amount','improvement_surcharge',
#                                     'total_amount','pickup_zip','dropoff_zip',
#                                       'pickup_dayofweek', 'pickup_hour', 'pickup_month','pickup_Queens','pickup_Brooklyn',
 #                                       'pickup_Manhattan','pickup_Bronx','dropoff_Queens','dropoff_Brooklyn',
 #                                       'dropoff_Manhattan','dropoff_Bronx','ptype_CCard','ptype_cash','ptype_nocharge',
  #                                      'ptype_dispute','ptype_voidtrip','RateCode_stdrate','RateCode_JFK',
   #                                     'RateCode_Newark','RateCode_NW','RateCode_Negotiated','RateCode_Group',
    #                                    'Duration'
=r1[0].toArray()
x[18]




array([ 0.02460368,  0.00459694, -0.00242413, -0.02030871, -0.01128363,
       -0.00815524,  0.01416242,  0.89698299,  0.00163576,  0.29461282,
       -0.02203972,  0.31885632, -0.09000388,  0.03929161, -0.06652559,
        0.64608975,  0.58869309,  0.01163316,  1.        ,  0.36270077,
        0.41295884, -0.01658767,  0.00465715,  0.02822905,  0.39567379,
        0.02765049, -0.34314617,  0.0011365 ,  0.32284087,  0.21704112,
       -0.41059884,  0.12499476,  0.19046307, -0.19082707, -0.00516868,
        0.00803957,         nan, -0.18506046,  0.15910981,  0.00852571,
        0.02194359,  0.10076259,         nan,  0.82628991])

In [101]:
pipeline_estimator = Pipeline(stages=[
    feature.VectorAssembler(inputCols=[ 'trip_distance','pickup_longitude','dropoff_longitude',
                                       'tip_amount','tolls_amount','pickup_zip','dropoff_zip',
                                        'pickup_dayofweek', 'pickup_hour', 'pickup_month','pickup_Queens','pickup_Brooklyn',
                                        'pickup_Manhattan','pickup_Bronx','dropoff_Queens','dropoff_Brooklyn',
                                        'dropoff_Manhattan','dropoff_Bronx','ptype_CCard','ptype_cash','ptype_nocharge',
                                        'ptype_dispute','ptype_voidtrip','RateCode_stdrate','RateCode_JFK',
                                        'RateCode_Newark','RateCode_NW','RateCode_Negotiated','RateCode_Group',
                                        'Duration'],
                           outputCol='features2'),
    regression.LinearRegression(featuresCol='features2', labelCol='total_amount')
])
pipe_model = pipeline_estimator.fit(features_sample_df)
pipe_model.transform(features_sample_df).show(3)

+----------+----+---+----+-------+--------+-------------------+-------------------+---------------+-------------+------------------+------------------+------------------+------------------+-----------------+-----------+-----+-------+----------+------------+---------------------+------------+----------+-------------------+-----------+--------------------+----------------+-------------------+-----------+------------+-------------+---------------+----------------+------------+--------------+----------------+-----------------+-------------+-----------+----------+--------------+-------------+--------------+----------------+------------+---------------+-----------+-------------------+--------------+--------+--------------------+--------------------+------------------+
|avg_temp_C|Rain|Fog|Snow|Holiday|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|store_and_fwd_flag| dropoff_longitude| dropoff_latitude|fare_amount|extra|mt

In [102]:
rmse1_df = pipe_model.transform(features_sample_df).select(fn.sqrt(fn.avg((fn.col('prediction')-fn.col('total_amount'))**2)).alias('rmse'))


In [103]:
rmse1_df.show()

+-----------------+
|             rmse|
+-----------------+
|2.646745033119248|
+-----------------+

